# Iteration 11: multiple reps (2)

## Plain English Summary

Reorganising the code so that it works with multiple replications is split over multiple steps.  The second ensures the a replication of the model returns results as opposed to printing them to the screen.

## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_11.md"))

modify CCUModel so that it returns the pandas dataframe instead of printing it out at the end of a run.

Only show the CCUModel class.  Do not hide any methods of CCUModel





## Date and time of input to LLM

> 1200 5th December 2023 

## Observations on the LLM Output

* The last line of `CCUModel.run()` returns the pandas data frame.

## LLM Output 1:

The reset_kpi method resets all the performance measures to their original values.

In [3]:
class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)
        self.performance_measures = None

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                if self.experiment.trace:
                    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.experiment.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        arrival_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            wait_time = self.env.now - arrival_time
            if self.experiment.trace:
                print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
                self.experiment.mean_waiting_time_unplanned += wait_time
                self.experiment.total_unplanned_admissions += 1
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def warmup_complete(self):
        yield self.env.timeout(self.experiment.warm_up_period)
        self.patient_count = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            mean_waiting_time_unplanned = self.experiment.mean_waiting_time_unplanned / self.experiment.total_unplanned_admissions
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds
            self.performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.experiment.cancelled_elective_count], 'Bed Utilization': [bed_utilization], 'Mean Waiting Time Unplanned': [mean_waiting_time_unplanned], 'Bed Occupancy': [bed_occupancy], 'Patient Count': [self.patient_count]})
            return self.performance_measures


## `Experiment` from prior iteration

In [4]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24,
                 trace=False):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.trace = trace
        self.patient_count = 0

    def reset_kpi(self):
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.patient_count = 0

In [5]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

,Cancelled Elective Operations,Bed Utilization,Mean Waiting Time Unplanned,Bed Occupancy,Patient Count
0,264,0.914601,27.159594,21.950421,1576


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [6]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [7]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [8]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [9]:
extreme_value_test_1()

### Test 2: Extreme value test 2

PASS

In [10]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means, trace=True)
    run_test(experiment)

In [11]:
try:
    extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective surgery patient 1 arrived at time 16.931617359450712
Elective surgery patient 1 admitted to critical care bed at time 16.931617359450712
Elective surgery patient 2 arrived at time 33.418747607053106
Elective surgery patient 2 admitted to critical care bed at time 33.418747607053106
Elective surgery patient 3 arrived at time 49.67912211733286
Elective surgery patient 3 admitted to critical care bed at time 49.67912211733286
Elective surgery patient 3 discharged from critical care bed at time 68.52806238082124
Elective surgery patient 3 discharged from critical care bed at time 69.13426628491857
Elective surgery patient 4 arrived at time 70.20048968988482
Elective surgery patient 4 admitted to critical care bed at time 70.20048968988482
Elective surgery patient 4 discharged from critical care bed at time 71.73898585298524
Critical care bed is available for next patient at time 73.52806238082124
Critical care bed is available for next patient at time 74.13426628491857
Critical ca

### Test 3: Different run length

PASS

In [12]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [13]:
try:
    test_results_collection_period(10)
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

### Test 4: Extreme value test 3

PASS.

In [14]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)
    print(f'{experiment.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')

In [15]:
extreme_value_test_3()

experiment.total_treatment_time=8708.682084216636
experiment.results_collection_period=8640


### Test 5: Extreme value test 4

PASS

In [16]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M, trace=True)
    run_test(experiment)

In [17]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Elective surgery patient 1 arrived at time 18.06954033769473
Elective surgery patient 1 admitted to critical care bed at time 18.06954033769473
Patient 2 arrived from the Wards at time 31.16202874837973
Patient 2 admitted to critical care bed at time 31.16202874837973
Elective surgery patient 3 arrived at time 38.10120423147018
Elective surgery patient 3 admitted to critical care bed at time 38.10120423147018
Patient 4 arrived from other hospitals at time 43.142738183222576
Patient 4 admitted to critical care bed at time 43.142738183222576
Patient 5 arrived from other hospitals at time 43.35558801405906
Patient 5 admitted to critical care bed at time 43.35558801405906
Elective surgery patient 6 arrived at time 54.44943907954905
Elective surgery patient 6 admitted to critical care bed at time 54.44943907954905
Patient 7 arrived from Emergency surgery at time 56.869986408838166
Patient 7 admitted to critical care bed at time 56.869986408838166
Patient 8 arrived from other hospitals at ti

### Test 6: Vary Warm-up parameter

PASS

In [18]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [19]:
test_warmup_period(0)

(8640, 8640)